# Dynamic Delta Hedging Simulation

This notebook covers:
1. Understanding delta hedging mechanics
2. Simulating stock price paths
3. Dynamic rehedging with different frequencies
4. Transaction costs impact
5. Gamma scalping P&L
6. Realized vs implied volatility comparison
7. Practical trading insights

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_delta,
    black_scholes_gamma
)
from options_desk.core.option import OptionType

## 1. Delta Hedging Theory

**Goal:** Eliminate directional risk from options position by trading the underlying.

**Process:**
1. Calculate option delta
2. Hold -Δ shares of stock (if long option)
3. Rehedge periodically as delta changes

**Key insight:** 
- Perfect continuous hedging → capture option premium minus theta
- Discrete hedging → transaction costs and gamma risk
- If realized vol < implied vol → hedge loses money
- If realized vol > implied vol → hedge makes money (gamma scalping)

## 2. Simulation Parameters

In [ ]:
# Market parameters
S0 = 100.0              # Initial stock price
K = 100.0               # Strike price (ATM)
T = 30 / 365.25         # 30 days to expiration
r = 0.045               # Risk-free rate
sigma_implied = 0.20    # Implied volatility (what we pay for option)
sigma_realized = 0.25   # Realized volatility (what actually happens)

# Simulation parameters
dt = 1 / 252            # Daily timestep (252 trading days/year)
n_steps = int(T / dt)   # Number of steps
n_paths = 1000          # Number of Monte Carlo paths

# Trading parameters
transaction_cost_bps = 5  # 5 basis points per trade
transaction_cost = transaction_cost_bps / 10000

# Option position
option_type = OptionType.CALL
position_size = 1  # Long 1 call option (100 shares)

print("Simulation Parameters:")
print(f"Initial Stock Price: ${S0:.2f}")
print(f"Strike: ${K:.2f}")
print(f"Days to Expiry: {T*365:.0f}")
print(f"Implied Vol: {sigma_implied*100:.1f}%")
print(f"Realized Vol: {sigma_realized*100:.1f}%")
print(f"Transaction Cost: {transaction_cost_bps} bps")
print(f"Number of Paths: {n_paths}")
print(f"Rehedge Frequency: Daily")

## 3. Simulate Stock Price Paths

Use Geometric Brownian Motion:
$$dS = \mu S dt + \sigma S dW$$

In [ ]:
def simulate_gbm(S0, mu, sigma, T, n_steps, n_paths, seed=42):
    """
    Simulate Geometric Brownian Motion paths.
    
    Returns:
    --------
    paths : ndarray of shape (n_paths, n_steps+1)
    """
    np.random.seed(seed)
    dt = T / n_steps
    
    # Generate random shocks
    Z = np.random.standard_normal((n_paths, n_steps))
    
    # Initialize paths
    paths = np.zeros((n_paths, n_steps + 1))
    paths[:, 0] = S0
    
    # Simulate paths
    for i in range(1, n_steps + 1):
        paths[:, i] = paths[:, i-1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, i-1]
        )
    
    return paths

# Simulate paths using realized volatility
stock_paths = simulate_gbm(S0, r, sigma_realized, T, n_steps, n_paths)

# Plot a few sample paths
fig, ax = plt.subplots(figsize=(14, 7))

time_grid = np.linspace(0, T * 365, n_steps + 1)

# Plot first 50 paths
for i in range(min(50, n_paths)):
    ax.plot(time_grid, stock_paths[i], alpha=0.3, linewidth=1)

# Plot mean path
mean_path = stock_paths.mean(axis=0)
ax.plot(time_grid, mean_path, 'r-', linewidth=3, label='Mean Path')

ax.axhline(K, color='black', linestyle='--', linewidth=2, label=f'Strike: ${K:.0f}')
ax.set_xlabel('Days', fontsize=12)
ax.set_ylabel('Stock Price ($)', fontsize=12)
ax.set_title(f'Simulated Stock Price Paths (σ = {sigma_realized*100:.0f}%)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Stock Prices:")
print(f"  Min: ${stock_paths[:, -1].min():.2f}")
print(f"  Mean: ${stock_paths[:, -1].mean():.2f}")
print(f"  Max: ${stock_paths[:, -1].max():.2f}")
print(f"  Std: ${stock_paths[:, -1].std():.2f}")

## 4. Delta Hedging Simulation

In [ ]:
def simulate_delta_hedging(stock_path, K, T, r, sigma_implied, option_type, 
                           transaction_cost=0.0005, hedge_frequency=1):
    """
    Simulate delta hedging for a single stock path.
    
    Parameters:
    -----------
    stock_path : array
        Simulated stock prices
    hedge_frequency : int
        Rehedge every N steps
    
    Returns:
    --------
    dict with hedging results
    """
    n_steps = len(stock_path) - 1
    dt_step = T / n_steps
    
    # Storage
    deltas = np.zeros(n_steps + 1)
    option_values = np.zeros(n_steps + 1)
    hedge_positions = np.zeros(n_steps + 1)
    hedge_pnl = np.zeros(n_steps + 1)
    transaction_costs = np.zeros(n_steps + 1)
    
    # Initial option purchase
    time_to_expiry = T
    initial_option_price = black_scholes_price(
        S0, K, time_to_expiry, r, sigma_implied, option_type
    )
    option_values[0] = initial_option_price
    
    # Initial delta and hedge
    deltas[0] = black_scholes_delta(
        S0, K, time_to_expiry, r, sigma_implied, option_type
    )
    hedge_positions[0] = -deltas[0] * 100  # Short delta shares to hedge
    transaction_costs[0] = abs(hedge_positions[0]) * S0 * transaction_cost
    
    # Simulate hedging over time
    for i in range(1, n_steps + 1):
        S = stock_path[i]
        time_to_expiry = T - i * dt_step
        
        if time_to_expiry > 0:
            # Calculate option value and delta
            option_values[i] = black_scholes_price(
                S, K, time_to_expiry, r, sigma_implied, option_type
            )
            deltas[i] = black_scholes_delta(
                S, K, time_to_expiry, r, sigma_implied, option_type
            )
        else:
            # At expiration
            if option_type == OptionType.CALL:
                option_values[i] = max(S - K, 0)
                deltas[i] = 1.0 if S > K else 0.0
            else:
                option_values[i] = max(K - S, 0)
                deltas[i] = -1.0 if S < K else 0.0
        
        # Rehedge if it's time
        if i % hedge_frequency == 0 or i == n_steps:
            new_hedge = -deltas[i] * 100
            hedge_change = new_hedge - hedge_positions[i-1]
            
            # P&L from hedge position
            hedge_pnl[i] = hedge_positions[i-1] * (S - stock_path[i-1])
            
            # Transaction cost
            transaction_costs[i] = abs(hedge_change) * S * transaction_cost
            
            hedge_positions[i] = new_hedge
        else:
            # No rehedging, just update P&L
            hedge_pnl[i] = hedge_positions[i-1] * (S - stock_path[i-1])
            hedge_positions[i] = hedge_positions[i-1]
    
    # Calculate total P&L
    option_pnl = option_values[-1] - option_values[0]
    total_hedge_pnl = hedge_pnl.sum()
    total_transaction_costs = transaction_costs.sum()
    total_pnl = option_pnl + total_hedge_pnl - total_transaction_costs
    
    return {
        'option_values': option_values,
        'deltas': deltas,
        'hedge_positions': hedge_positions,
        'hedge_pnl': hedge_pnl,
        'transaction_costs': transaction_costs,
        'option_pnl': option_pnl,
        'total_hedge_pnl': total_hedge_pnl,
        'total_transaction_costs': total_transaction_costs,
        'total_pnl': total_pnl
    }

# Run simulation for first path
result = simulate_delta_hedging(
    stock_paths[0], K, T, r, sigma_implied, option_type, transaction_cost
)

print(f"\nHedging Results for Path 1:")
print(f"  Initial Option Price: ${result['option_values'][0]:.2f}")
print(f"  Final Option Value: ${result['option_values'][-1]:.2f}")
print(f"  Option P&L: ${result['option_pnl']:.2f}")
print(f"  Hedge P&L: ${result['total_hedge_pnl']:.2f}")
print(f"  Transaction Costs: ${result['total_transaction_costs']:.2f}")
print(f"  Total P&L: ${result['total_pnl']:.2f}")

## 5. Visualize Delta Hedging for Single Path

In [ ]:
# Create detailed visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

time_grid = np.linspace(0, T * 365, n_steps + 1)

# Stock price and option value
ax1 = axes[0, 0]
ax1_twin = ax1.twinx()
ax1.plot(time_grid, stock_paths[0], 'b-', linewidth=2, label='Stock Price')
ax1_twin.plot(time_grid, result['option_values'], 'r-', linewidth=2, label='Option Value')
ax1.axhline(K, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Strike')
ax1.set_xlabel('Days')
ax1.set_ylabel('Stock Price ($)', color='b')
ax1_twin.set_ylabel('Option Value ($)', color='r')
ax1.set_title('Stock Price and Option Value', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Delta evolution
axes[0, 1].plot(time_grid, result['deltas'], 'g-', linewidth=2)
axes[0, 1].axhline(0.5, color='black', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_xlabel('Days')
axes[0, 1].set_ylabel('Delta')
axes[0, 1].set_title('Delta Evolution', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Hedge position
axes[1, 0].plot(time_grid, result['hedge_positions'], 'purple', linewidth=2)
axes[1, 0].axhline(0, color='black', linestyle='-', linewidth=1)
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Shares')
axes[1, 0].set_title('Hedge Position (Short Shares)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Cumulative P&L
cumulative_pnl = np.cumsum(result['hedge_pnl']) - np.cumsum(result['transaction_costs'])
axes[1, 1].plot(time_grid, cumulative_pnl, 'orange', linewidth=2)
axes[1, 1].axhline(0, color='black', linestyle='-', linewidth=1)
axes[1, 1].set_xlabel('Days')
axes[1, 1].set_ylabel('Cumulative P&L ($)')
axes[1, 1].set_title('Cumulative Hedge P&L (net of costs)', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Monte Carlo Simulation - All Paths

In [ ]:
# Run simulation for all paths
print("Running Monte Carlo simulation...")

all_pnls = []
all_option_pnls = []
all_hedge_pnls = []
all_transaction_costs = []

for i in range(n_paths):
    result = simulate_delta_hedging(
        stock_paths[i], K, T, r, sigma_implied, option_type, transaction_cost
    )
    all_pnls.append(result['total_pnl'])
    all_option_pnls.append(result['option_pnl'])
    all_hedge_pnls.append(result['total_hedge_pnl'])
    all_transaction_costs.append(result['total_transaction_costs'])

all_pnls = np.array(all_pnls)
all_option_pnls = np.array(all_option_pnls)
all_hedge_pnls = np.array(all_hedge_pnls)
all_transaction_costs = np.array(all_transaction_costs)

print("\n" + "="*70)
print("MONTE CARLO HEDGING RESULTS")
print("="*70)
print(f"\nImplied Vol: {sigma_implied*100:.1f}% (what we paid)")
print(f"Realized Vol: {sigma_realized*100:.1f}% (what actually happened)")
print(f"Vol Difference: {(sigma_realized - sigma_implied)*100:.1f}%")

print(f"\nOption P&L:")
print(f"  Mean: ${all_option_pnls.mean():.2f}")
print(f"  Std: ${all_option_pnls.std():.2f}")

print(f"\nHedge P&L:")
print(f"  Mean: ${all_hedge_pnls.mean():.2f}")
print(f"  Std: ${all_hedge_pnls.std():.2f}")

print(f"\nTransaction Costs:")
print(f"  Mean: ${all_transaction_costs.mean():.2f}")
print(f"  Total across all paths: ${all_transaction_costs.sum():.2f}")

print(f"\nNet P&L (Option + Hedge - Costs):")
print(f"  Mean: ${all_pnls.mean():.2f}")
print(f"  Std: ${all_pnls.std():.2f}")
print(f"  Min: ${all_pnls.min():.2f}")
print(f"  Max: ${all_pnls.max():.2f}")
print(f"  Profitable paths: {(all_pnls > 0).sum()} / {n_paths} ({(all_pnls > 0).mean()*100:.1f}%)")

# Interpretation
print(f"\n" + "="*70)
print("INTERPRETATION")
print("="*70)
if sigma_realized > sigma_implied:
    print(f"Realized vol > Implied vol → Gamma scalping profits")
    print(f"We bought the option too cheap and made money from rehedging")
else:
    print(f"Realized vol < Implied vol → Hedging loses money")
    print(f"We overpaid for the option and lost money from rehedging")

## 7. P&L Distribution

In [ ]:
# Plot P&L distributions
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Total P&L distribution
axes[0].hist(all_pnls, bins=50, alpha=0.7, edgecolor='black', color='blue')
axes[0].axvline(all_pnls.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: ${all_pnls.mean():.2f}')
axes[0].axvline(0, color='black', linestyle='-', linewidth=1)
axes[0].set_xlabel('P&L ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Delta Hedged P&L Distribution', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Component breakdown
components = [
    ('Option P&L', all_option_pnls, 'green'),
    ('Hedge P&L', all_hedge_pnls, 'blue'),
    ('Transaction Costs', -all_transaction_costs, 'red'),
    ('Net P&L', all_pnls, 'black')
]

for name, data, color in components:
    axes[1].hist(data, bins=30, alpha=0.4, label=name, color=color)

axes[1].axvline(0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('P&L ($)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('P&L Component Distributions', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Compare Different Hedging Frequencies

In [ ]:
# Test different rehedging frequencies
hedge_frequencies = [1, 2, 5, 10]  # Daily, every 2 days, weekly, etc.
frequency_results = {}

print("Testing different hedging frequencies...\n")

for freq in hedge_frequencies:
    pnls = []
    costs = []
    
    for i in range(min(100, n_paths)):  # Use subset for speed
        result = simulate_delta_hedging(
            stock_paths[i], K, T, r, sigma_implied, option_type, 
            transaction_cost, hedge_frequency=freq
        )
        pnls.append(result['total_pnl'])
        costs.append(result['total_transaction_costs'])
    
    frequency_results[freq] = {
        'mean_pnl': np.mean(pnls),
        'std_pnl': np.std(pnls),
        'mean_cost': np.mean(costs)
    }
    
    print(f"Rehedge every {freq} day(s):")
    print(f"  Mean P&L: ${np.mean(pnls):>7.2f}")
    print(f"  Std P&L:  ${np.std(pnls):>7.2f}")
    print(f"  Avg Cost: ${np.mean(costs):>7.2f}")
    print()

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

freqs = list(frequency_results.keys())
mean_pnls = [frequency_results[f]['mean_pnl'] for f in freqs]
std_pnls = [frequency_results[f]['std_pnl'] for f in freqs]
mean_costs = [frequency_results[f]['mean_cost'] for f in freqs]

# Mean P&L vs frequency
axes[0].plot(freqs, mean_pnls, 'o-', linewidth=2, markersize=8)
axes[0].axhline(0, color='black', linestyle='--', linewidth=1)
axes[0].set_xlabel('Rehedge Frequency (days)', fontsize=12)
axes[0].set_ylabel('Mean P&L ($)', fontsize=12)
axes[0].set_title('Mean P&L vs Hedging Frequency', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Transaction costs vs frequency
axes[1].plot(freqs, mean_costs, 'o-', linewidth=2, markersize=8, color='red')
axes[1].set_xlabel('Rehedge Frequency (days)', fontsize=12)
axes[1].set_ylabel('Average Transaction Costs ($)', fontsize=12)
axes[1].set_title('Transaction Costs vs Hedging Frequency', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("- More frequent hedging → Lower variance but higher transaction costs")
print("- Less frequent hedging → Higher variance but lower transaction costs")
print("- Optimal frequency depends on gamma exposure and transaction costs")

## 9. Realized vs Implied Volatility Impact

In [ ]:
# Test different realized volatilities
realized_vols = np.linspace(0.10, 0.40, 10)
vol_comparison_results = []

print("Testing different realized volatilities...\n")

for sigma_real in realized_vols:
    # Simulate new paths
    paths = simulate_gbm(S0, r, sigma_real, T, n_steps, 100, seed=42)
    
    pnls = []
    for i in range(len(paths)):
        result = simulate_delta_hedging(
            paths[i], K, T, r, sigma_implied, option_type, transaction_cost
        )
        pnls.append(result['total_pnl'])
    
    vol_comparison_results.append({
        'realized_vol': sigma_real,
        'mean_pnl': np.mean(pnls),
        'std_pnl': np.std(pnls)
    })

vol_df = pd.DataFrame(vol_comparison_results)

# Plot
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(vol_df['realized_vol'] * 100, vol_df['mean_pnl'], 'o-', 
        linewidth=2, markersize=8, color='blue')
ax.axvline(sigma_implied * 100, color='red', linestyle='--', linewidth=2, 
           label=f'Implied Vol = {sigma_implied*100:.0f}%')
ax.axhline(0, color='black', linestyle='-', linewidth=1)

ax.fill_between(vol_df['realized_vol'] * 100, 
                vol_df['mean_pnl'] - vol_df['std_pnl'],
                vol_df['mean_pnl'] + vol_df['std_pnl'],
                alpha=0.3, label='±1 Std Dev')

ax.set_xlabel('Realized Volatility (%)', fontsize=12)
ax.set_ylabel('Mean P&L ($)', fontsize=12)
ax.set_title('Delta Hedged P&L vs Realized Volatility', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print(f"- If realized vol < {sigma_implied*100:.0f}% (implied): Hedge loses money")
print(f"- If realized vol > {sigma_implied*100:.0f}% (implied): Hedge makes money (gamma scalping)")
print(f"- P&L is roughly linear in the vol difference")
print(f"- This is the core of volatility trading!")

## 10. Summary

### Key Takeaways:

1. **Delta Hedging Mechanics:**
   - Eliminates directional risk by trading underlying
   - Requires frequent rebalancing as delta changes
   - P&L depends on realized vs implied volatility difference

2. **Transaction Costs:**
   - Significant drag on returns with frequent rehedging
   - Trade-off between hedging precision and costs
   - Optimal frequency depends on gamma and cost structure

3. **Gamma Scalping:**
   - When realized vol > implied vol → hedge profits
   - When realized vol < implied vol → hedge loses
   - This is how market makers profit from volatility

4. **Practical Implications:**
   - Cannot perfectly hedge in discrete time
   - Gamma risk remains between rehedges
   - Vega and theta still affect portfolio

### Next Notebook:
**05_backtest.ipynb** - Backtest option strategies on historical data with realistic execution assumptions.